In [1]:
# !pip install openpyxl

In [2]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import re

In [3]:
data = pd.read_csv('data.csv')

In [4]:
date_group = []
for date in data['date']:
    if date in range(1, 11):
        date_group.append('Begin')
    elif date in range(11, 21):
        date_group.append('Middle')
    else:
        date_group.append('End')

data.insert(data.columns.get_loc('date') + 1, 'date group', date_group)

In [5]:
month_group = []
for month in data['month']:
    if month in range(1, 4):
        month_group.append('Q1')
    elif month in range(4, 7):
        month_group.append('Q2')
    elif month in range(7, 10):
        month_group.append('Q3')
    else:
        month_group.append('Q4')

data.insert(data.columns.get_loc('month') + 1, 'month group', month_group)

In [6]:
weekday_group = []
for weekday in data['weekday']:
    if weekday in ['SATURDAY', 'SUNDAY']:
        weekday_group.append('Weekend')
    else:
        weekday_group.append('Weekday')

data.insert(data.columns.get_loc('weekday') + 1, 'weekday group', weekday_group)

In [7]:
nights_group = []
for nights in data['nights']:
    if nights in range(1, 4):
        nights_group.append('1-3')
    elif nights in range(4, 7):
        nights_group.append('4-6')
    elif nights in range(7, 10):
        nights_group.append('7-9')
    elif nights in range(10, 13):
        nights_group.append('10-12')
    else:
        nights_group.append('12 longer')

data.insert(data.columns.get_loc('nights') + 1, 'nights group', nights_group)

In [8]:
age_group = []
for age in data['age_correct']:
    if age in range(17, 20):
        age_group.append('under 20')
    elif age in range(20, 30):
        age_group.append('20s')
    elif age in range(30, 40):
        age_group.append('30s')
    elif age in range(40, 50):
        age_group.append('40s')
    elif age in range(50, 60):
        age_group.append('50s')
    elif age in range(60, 70):
        age_group.append('60s')
    else:
        age_group.append('70 older')

data.insert(data.columns.get_loc('age_correct') + 1, 'age group', age_group)

In [9]:
country_group = []
for country in data['country']:
    if country in ['Belgium', 'Netherlands', 'France']:
        country_group.append('Western EU')
    else:
        country_group.append('Southern EU')

data.insert(data.columns.get_loc('country') + 1, 'country group', country_group)

In [10]:
data

,Column,hotel,country,country group,arrival,date,date group,month,month group,weekday,weekday group,nights,nights group,room,price,age,age_correct,age group,gender
0,0,resort,Belgium,Western EU,2017-02-20,20,Middle,2,Q1,MONDAY,Weekday,1,1-3,double,45,59,59,50s,f
1,1,city,Portugal,Southern EU,2017-09-28,28,End,9,Q3,THURSDAY,Weekday,3,1-3,double,211,32,32,30s,f
2,2,city,Italy,Southern EU,2019-07-14,14,Middle,7,Q3,SUNDAY,Weekend,5,4-6,double,330,1978,41,40s,m
3,3,city,Portugal,Southern EU,2019-08-02,2,Begin,8,Q3,FRIDAY,Weekday,4,4-6,single,284,67,67,60s,f
4,4,resort,Netherlands,Western EU,2019-09-26,26,End,9,Q3,THURSDAY,Weekday,2,1-3,double,147,60,60,60s,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,city,Spain,Southern EU,2017-11-03,3,Begin,11,Q4,FRIDAY,Weekday,3,1-3,double,283,31,31,30s,m
996,996,city,Belgium,Western EU,2019-01-30,30,End,1,Q1,WEDNESDAY,Weekday,4,4-6,double,204,24,24,20s,m
997,997,resort,Spain,Southern EU,2017-07-03,3,Begin,7,Q3,MONDAY,Weekday,6,4-6,double,486,25,25,20s,?
998,998,resort,Spain,Southern EU,2018-04-09,9,Begin,4,Q2,MONDAY,Weekday,5,4-6,double,405,48,48,40s,?


In [11]:
year = [re.findall('[0-9]{4}', date)[0] for date in data['arrival']]

In [12]:
data['year'] = year

In [13]:
analysis = data.drop(['Column', 'arrival', 'age'], axis = 1)

In [14]:
analysis

,hotel,country,country group,date,date group,month,month group,weekday,weekday group,nights,nights group,room,price,age_correct,age group,gender,year
0,resort,Belgium,Western EU,20,Middle,2,Q1,MONDAY,Weekday,1,1-3,double,45,59,50s,f,2017
1,city,Portugal,Southern EU,28,End,9,Q3,THURSDAY,Weekday,3,1-3,double,211,32,30s,f,2017
2,city,Italy,Southern EU,14,Middle,7,Q3,SUNDAY,Weekend,5,4-6,double,330,41,40s,m,2019
3,city,Portugal,Southern EU,2,Begin,8,Q3,FRIDAY,Weekday,4,4-6,single,284,67,60s,f,2019
4,resort,Netherlands,Western EU,26,End,9,Q3,THURSDAY,Weekday,2,1-3,double,147,60,60s,m,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,city,Spain,Southern EU,3,Begin,11,Q4,FRIDAY,Weekday,3,1-3,double,283,31,30s,m,2017
996,city,Belgium,Western EU,30,End,1,Q1,WEDNESDAY,Weekday,4,4-6,double,204,24,20s,m,2019
997,resort,Spain,Southern EU,3,Begin,7,Q3,MONDAY,Weekday,6,4-6,double,486,25,20s,?,2017
998,resort,Spain,Southern EU,9,Begin,4,Q2,MONDAY,Weekday,5,4-6,double,405,48,40s,?,2018


In [15]:
analysis.dtypes

hotel            object
country          object
country group    object
date              int64
date group       object
month             int64
month group      object
weekday          object
weekday group    object
nights            int64
nights group     object
room             object
price             int64
age_correct       int64
age group        object
gender           object
year             object
dtype: object

In [16]:
def reorder(table):
    cols = list(table.columns)
    cols = [cols[-1]] + cols[:-1]
    table = table[cols]
    table = table.rename(index = {'All': 'Base'})
    return table

In [17]:
from statsmodels.stats.proportion import proportions_ztest
    
def ztesting(table_n):
    
    table_result = table_n.copy()

    for column in range(0, len(table_n.columns)):
        result_list = [table_n[table_n.columns[column]][0]]

        for row in range(1, len(table_n)):            
                result_set = []

                for column_cal in range(0, len(table_n.columns)):
                    if not table_n.columns[column] == table_n.columns[column_cal]:

                        z_score, p_value = proportions_ztest([
                            table_n[table_n.columns[column]][row], table_n[table_n.columns[column_cal]][row]
                        ], [
                            table_n[table_n.columns[column]][0], table_n[table_n.columns[column_cal]][0]
                        ])
                        
                        p1 = table_n[table_n.columns[column]][row] / table_n[table_n.columns[column]][0]
                        p2 = table_n[table_n.columns[column_cal]][row] / table_n[table_n.columns[column_cal]][0]

                        if p_value < 0.05:
                            if max([p1, p2]) == p1:
                                result_set.append(column_cal)

                if len(result_set) > 0:
                    result_set = [str(result) for result in result_set]
                    result_string = ', '.join(result_set)
                    result_list.append(result_string)
                else:
                    result_list.append('')

        table_result[table_result.columns[column]] = result_list
    
    return table_result

In [18]:
def table_export(file_name, analysis):
    with pd.ExcelWriter(file_name) as writer:
        column = list(analysis.columns)

        for i in range(0, len(column)):
            row = 0
            for c in range(0, len(column)):
                if not column[i] == column[c]:
                    if not analysis[column[i]].dtypes == 'int64':
                        if not analysis[column[c]].dtypes == 'int64':

                            table_p = pd.crosstab(
                                analysis[column[c]], analysis[column[i]], normalize = 'columns', margins = True
                            ).sort_values(by = ['All'], ascending = False).round(4)*100
                            table_p = reorder(table_p)

                            table_n = pd.crosstab(
                                analysis[column[c]], analysis[column[i]], margins = True
                            ).sort_values(by = ['All'], ascending = False)
                            table_n = reorder(table_n)

                            row_to_insert = table_n.loc['Base']
                            table_p = pd.concat([table_p.iloc[:0], row_to_insert.to_frame().T, table_p.iloc[0:]])

                            table_p.to_excel(writer, sheet_name = column[i], startrow = row)

                            table_test = ztesting(table_n)
                            table_test.to_excel(writer, sheet_name = column[i], startrow = row, startcol = len(table_p.columns) + 2)

                            row += len(table_p) + 2

In [21]:
table_export('analysis.xlsx', analysis)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:790: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
C:\ProgramData\Anaconda3\lib

In [20]:
# n1 = 10000
# n2 = 12000
# successes1 = 10
# successes2 = 27

# prop1 = successes1 / n1
# prop2 = successes2 / n2

# z_score, p_value = proportions_ztest([successes1, successes2], [n1, n2])

# print(f'Z-score: {z_score:.3f}')
# print(f'p-value: {p_value:.3f}')

# if p_value < 0.05:
#     print('差異顯著')
# else:
#     print('差異不顯著')